# 1 有监督分类


## 1.1 性别鉴定

在4中，我们看到，男性和女性的名字有一些鲜明的特点。以a，e和i结尾的很可能是女性，而以k，o，r，s和t结尾的很可能是男性。让我们建立一个分类器更精确地模拟这些差异。

创建一个分类器的第一步是决定输入的什么样的特征是相关的，以及如何为那些特征编码。在这个例子中，我们一开始只是寻找一个给定的名称的最后一个字母。以下特征提取器函数建立一个字典，包含有关给定名称的相关信息：

In [1]:
def gender_features(word):
    return {"last_letter": word[-1]}
gender_features("Shrek")

{'last_letter': 'k'}

现在，我们已经定义了一个特征提取器，我们需要准备一个例子和对应类标签的列表。

In [4]:
from nltk.corpus import names
labeled_names = ([(name, "male") for name in names.words("male.txt")]) + \
[(name, "female") for name in names.words("female.txt")]
labeled_names

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abbey', 'male'),
 ('Abbie', 'male'),
 ('Abbot', 'male'),
 ('Abbott', 'male'),
 ('Abby', 'male'),
 ('Abdel', 'male'),
 ('Abdul', 'male'),
 ('Abdulkarim', 'male'),
 ('Abdullah', 'male'),
 ('Abe', 'male'),
 ('Abel', 'male'),
 ('Abelard', 'male'),
 ('Abner', 'male'),
 ('Abraham', 'male'),
 ('Abram', 'male'),
 ('Ace', 'male'),
 ('Adair', 'male'),
 ('Adam', 'male'),
 ('Adams', 'male'),
 ('Addie', 'male'),
 ('Adger', 'male'),
 ('Aditya', 'male'),
 ('Adlai', 'male'),
 ('Adnan', 'male'),
 ('Adolf', 'male'),
 ('Adolfo', 'male'),
 ('Adolph', 'male'),
 ('Adolphe', 'male'),
 ('Adolpho', 'male'),
 ('Adolphus', 'male'),
 ('Adrian', 'male'),
 ('Adrick', 'male'),
 ('Adrien', 'male'),
 ('Agamemnon', 'male'),
 ('Aguinaldo', 'male'),
 ('Aguste', 'male'),
 ('Agustin', 'male'),
 ('Aharon', 'male'),
 ('Ahmad', 'male'),
 ('Ahmed', 'male'),
 ('Ahmet', 'male'),
 ('Ajai', 'male'),
 ('Ajay', 'male'),
 ('Al', 'male'),
 ('Alaa', 'male'),
 ('Alain', 'male'),
 ('Alan', 'male

In [5]:
import random
random.shuffle(labeled_names)

接下来，我们使用特征提取器处理names数据，并划分特征集的结果链表为一个训练集和一个测试集。训练集用于训练一个新的“朴素贝叶斯”分类器。

In [10]:
import nltk
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

在本章的后面，我们将学习更多关于朴素贝叶斯分类器的内容。现在，让我们只是在上面测试一些没有出现在训练数据中的名字：

In [6]:
print(classifier.classify(gender_features("Neo")))
print(classifier.classify(gender_features("Trinity")))

male
female


请看《黑客帝国》中这些角色的名字被正确分类。尽管这部科幻电影的背景是在2199年，但它仍然符合我们有关名字和性别的预期。我们可以在大数据量的未见过的数据上系统地评估这个分类器：

In [7]:
print(nltk.classify.accuracy(classifier, test_set))

0.76


最后，我们可以检查分类器，确定哪些特征对于区分名字的性别是最有效的：

In [8]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'k'              male : female =     74.4 : 1.0
             last_letter = 'a'            female : male   =     39.7 : 1.0
             last_letter = 'f'              male : female =     22.2 : 1.0
             last_letter = 'p'              male : female =     11.9 : 1.0
             last_letter = 'd'              male : female =      9.6 : 1.0


此列表显示训练集中以a 结尾的名字中女性是男性的38倍，而以k 结尾名字中男性是女性的31倍。这些比率称为似然比，可以用于比较不同特征-结果关系。

在处理大型语料库时，构建一个包含每一个实例的特征的单独的列表会使用大量的内存。在这些情况下，使用函数nltk.classify.apply_features，返回一个行为像一个列表而不会在内存存储所有特征集的对象：

In [9]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

## 1.2 选择正确的特征

选择相关的特征，并决定如何为一个学习方法编码它们，这对学习方法提取一个好的模型可以产生巨大的影响。建立一个分类器的很多有趣的工作之一是找出哪些特征可能是相关的，以及我们如何能够表示它们。虽然使用相当简单而明显的特征集往往可以得到像样的性能，但是使用精心构建的基于对当前任务的透彻理解的特征，通常会显著提高收益。

典型地，特征提取通过反复试验和错误的过程建立的，由哪些信息是与问题相关的直觉指引的。它通常以“厨房水槽”的方法开始，包括你能想到的所有特征，然后检查哪些特征是实际有用的。我们在1.2中对名字性别特征采取这种做法。

In [10]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in "abcdefghijklmnopqrstuvwxyz":
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [11]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))         # 书上这样说的低1%，可是我的却比0.75高了3个百分点

0.746


In [12]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

已经将语料分为适当的数据集，我们使用训练集训练一个模型，然后在开发测试集上运行。

In [13]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.766


使用开发测试集，我们可以生成一个分类器预测名字性别时的错误列表：

In [14]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

然后，可以检查个别错误案例，在那里该模型预测了错误的标签，尝试确定什么额外信息将使其能够作出正确的决定（或者现有的哪部分信息导致其做出错误的决定）。然后可以相应的调整特征集。我们已经建立的名字分类器在开发测试语料上产生约100个错误：

In [15]:
count = 0
for (tag, guess, name) in sorted(errors):
    print("correct={:<8} guess={:<8s} name={:<30}".format(tag, guess, name))
    count += 1
    if count >10:
        break

correct=female   guess=male     name=Abigael                       
correct=female   guess=male     name=Addis                         
correct=female   guess=male     name=Adrien                        
correct=female   guess=male     name=Amabel                        
correct=female   guess=male     name=Anet                          
correct=female   guess=male     name=Ardis                         
correct=female   guess=male     name=Arlen                         
correct=female   guess=male     name=Astrid                        
correct=female   guess=male     name=Averyl                        
correct=female   guess=male     name=Beret                         
correct=female   guess=male     name=Bren                          


浏览这个错误列表，它明确指出一些多个字母的后缀可以指示名字性别。例如，yn结尾的名字显示以女性为主，尽管事实上，n结尾的名字往往是男性；以ch结尾的名字通常是男性，尽管以h结尾的名字倾向于是女性。因此，调整我们的特征提取器包括两个字母后缀的特征：

In [16]:
def gender_features(word):
    return {"suffix1": word[-1:],
           "suffix2": word[-2:]}

使用新的特征提取器重建分类器，我们看到测试数据集上的性能提高了近3个百分点（从76.5％到79.6％）：

In [17]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.793


## 1.3 文档分类

在1中，我们看到了语料库的几个例子，那里文档已经按类别标记。使用这些语料库，我们可以建立分类器，自动给新文档添加适当的类别标签。首先，我们构造一个标记了相应类别的文档清单。对于这个例子，我们选择电影评论语料库，将每个评论归类为正面或负面。

In [6]:
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
                         for category in movie_reviews.categories()
                         for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [8]:
documents[0]

(['the',
  'summer',
  'movie',
  'season',
  'is',
  'always',
  'the',
  'biggest',
  'and',
  'brightest',
  'season',
  'for',
  'movies',
  'throughout',
  'the',
  'year',
  '.',
  'we',
  'are',
  'introduced',
  'to',
  'blockbuster',
  'hits',
  'that',
  'are',
  'usually',
  'trashed',
  'by',
  'critics',
  ',',
  'loved',
  'by',
  'the',
  'people',
  ',',
  'and',
  'make',
  'tons',
  'of',
  'money',
  'at',
  'the',
  'box',
  'office',
  '.',
  'this',
  'year',
  ',',
  'the',
  'tradition',
  'continues',
  'with',
  'an',
  'update',
  'on',
  'the',
  'universal',
  '1932',
  'classic',
  ',',
  'the',
  'mummy',
  '.',
  'i',
  'have',
  'always',
  'been',
  'a',
  'sucker',
  'for',
  'this',
  'kind',
  'of',
  'monster',
  'movies',
  ',',
  'and',
  'i',
  'always',
  'seem',
  'to',
  'be',
  'the',
  'only',
  'critic',
  'who',
  'really',
  'likes',
  'them',
  '.',
  'that',
  'tradition',
  'continues',
  'as',
  'well',
  'with',
  'this',
  'fun',
 

In [13]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
all_words

FreqDist({',': 77717, 'the': 76529, '.': 65876, 'a': 38106, 'and': 35576, 'of': 34123, 'to': 31937, "'": 30585, 'is': 25195, 'in': 21822, ...})

In [14]:
word_features = list(all_words.keys())[:2000]
word_features

['plot',
 ':',
 'two',
 'teen',
 'couples',
 'go',
 'to',
 'a',
 'church',
 'party',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.',
 'they',
 'get',
 'into',
 'an',
 'accident',
 'one',
 'of',
 'the',
 'guys',
 'dies',
 'but',
 'his',
 'girlfriend',
 'continues',
 'see',
 'him',
 'in',
 'her',
 'life',
 'has',
 'nightmares',
 'what',
 "'",
 's',
 'deal',
 '?',
 'watch',
 'movie',
 '"',
 'sorta',
 'find',
 'out',
 'critique',
 'mind',
 '-',
 'fuck',
 'for',
 'generation',
 'that',
 'touches',
 'on',
 'very',
 'cool',
 'idea',
 'presents',
 'it',
 'bad',
 'package',
 'which',
 'is',
 'makes',
 'this',
 'review',
 'even',
 'harder',
 'write',
 'since',
 'i',
 'generally',
 'applaud',
 'films',
 'attempt',
 'break',
 'mold',
 'mess',
 'with',
 'your',
 'head',
 'such',
 '(',
 'lost',
 'highway',
 '&',
 'memento',
 ')',
 'there',
 'are',
 'good',
 'ways',
 'making',
 'all',
 'types',
 'these',
 'folks',
 'just',
 'didn',
 't',
 'snag',
 'correctly',
 'seem',
 'have',
 'taken',
 'pretty',


In [ ]:
# 简单地检查这些词是否在一个给定的文档中
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features["contains({})".format(word)] = (word in document_words)
    return features

In [15]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.75


In [16]:
classifier.show_most_informative_features(5) 

Most Informative Features
 contains(unimaginative) = True              neg : pos    =      8.4 : 1.0
        contains(welles) = True              neg : pos    =      8.4 : 1.0
    contains(schumacher) = True              neg : pos    =      7.5 : 1.0
          contains(mena) = True              neg : pos    =      7.1 : 1.0
        contains(shoddy) = True              neg : pos    =      7.1 : 1.0


## 1.4 词性标注

第5.中，我们建立了一个正则表达式标注器，通过查找词内部的组成，为词选择词性标记。然而，这个正则表达式标注器是手工制作的。作为替代，我们可以训练一个分类器来算出哪个后缀最有信息量。首先，让我们找出最常见的后缀：

In [17]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [18]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
common_suffixes

['e',
 ',',
 '.',
 's',
 'd',
 't',
 'he',
 'n',
 'a',
 'of',
 'the',
 'y',
 'r',
 'to',
 'in',
 'f',
 'o',
 'ed',
 'nd',
 'is',
 'on',
 'l',
 'g',
 'and',
 'ng',
 'er',
 'as',
 'ing',
 'h',
 'at',
 'es',
 'or',
 're',
 'it',
 '``',
 'an',
 "''",
 'm',
 ';',
 'i',
 'ly',
 'ion',
 'en',
 'al',
 '?',
 'nt',
 'be',
 'hat',
 'st',
 'his',
 'th',
 'll',
 'le',
 'ce',
 'by',
 'ts',
 'me',
 've',
 "'",
 'se',
 'ut',
 'was',
 'for',
 'ent',
 'ch',
 'k',
 'w',
 'ld',
 '`',
 'rs',
 'ted',
 'ere',
 'her',
 'ne',
 'ns',
 'ith',
 'ad',
 'ry',
 ')',
 '(',
 'te',
 '--',
 'ay',
 'ty',
 'ot',
 'p',
 'nce',
 "'s",
 'ter',
 'om',
 'ss',
 ':',
 'we',
 'are',
 'c',
 'ers',
 'uld',
 'had',
 'so',
 'ey']

接下来，我们将定义一个特征提取器函数，检查给定的单词的这些后缀：

In [19]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features["endswith({})".format(suffix)] = word.lower().endswith(suffix)
    return features

In [20]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]
featuresets

[({'endswith(e)': True,
   'endswith(,)': False,
   'endswith(.)': False,
   'endswith(s)': False,
   'endswith(d)': False,
   'endswith(t)': False,
   'endswith(he)': True,
   'endswith(n)': False,
   'endswith(a)': False,
   'endswith(of)': False,
   'endswith(the)': True,
   'endswith(y)': False,
   'endswith(r)': False,
   'endswith(to)': False,
   'endswith(in)': False,
   'endswith(f)': False,
   'endswith(o)': False,
   'endswith(ed)': False,
   'endswith(nd)': False,
   'endswith(is)': False,
   'endswith(on)': False,
   'endswith(l)': False,
   'endswith(g)': False,
   'endswith(and)': False,
   'endswith(ng)': False,
   'endswith(er)': False,
   'endswith(as)': False,
   'endswith(ing)': False,
   'endswith(h)': False,
   'endswith(at)': False,
   'endswith(es)': False,
   'endswith(or)': False,
   'endswith(re)': False,
   'endswith(it)': False,
   'endswith(``)': False,
   'endswith(an)': False,
   "endswith('')": False,
   'endswith(m)': False,
   'endswith(;)': False,
   

In [21]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6270512182993535

In [22]:
classifier.classify(pos_features("cats"))

'NNS'

决策树模型的一个很好的性质是它们往往很容易解释——我们甚至可以指示NLTK将它们以伪代码形式输出：

In [24]:
print(classifier.pseudocode(depth = 4))

if endswith(the) == False: 
  if endswith(,) == False: 
    if endswith(s) == False: 
      if endswith(.) == False: return '.'
      if endswith(.) == True: return '.'
    if endswith(s) == True: 
      if endswith(is) == False: return 'PP$'
      if endswith(is) == True: return 'BEZ'
  if endswith(,) == True: return ','
if endswith(the) == True: return 'AT'



## 1.5 探索上下文语境

In [25]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
               "suffix(2)": sentence[i][-2:],
               "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i - 1]
    return features

pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [26]:
tagged_sents = brown.tagged_sents(categories='news')
tagged_sents

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

In [27]:
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag) )
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

## 1.6 序列分类

In [28]:
def pos_features(sentence, i, history):
    features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i - 1]
        features["prev-tag"] = history[i - 1]
    return features

class ConsecutivePosTagger(nltk.TaggerI): 
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [30]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)


In [31]:
tagger.evaluate(test_sents)

C:\Users\raoxinyu\AppData\Local\Temp\ipykernel_21692\2463806165.py:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tagger.evaluate(test_sents)


0.7980528511821975

# 2 有监督分类的更多例子
## 2.1 句子分割

句子分割可以看作是一个标点符号的分类任务：每当我们遇到一个可能会结束一个句子的符号，如句号或问号，我们必须决定它是否终止了当前句子。

第一步是获得一些已被分割成句子的数据，将它转换成一种适合提取特征的形式：

In [32]:
sents = nltk.corpus.treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset - 1)

在这里，tokens是单独句子标识符的合并列表，boundaries是一个包含所有句子边界词符索引的集合。下一步，我们需要指定用于决定标点是否表示句子边界的数据特征：

In [33]:
def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i+1][0].isupper(),
             'prev-word': tokens[i-1].lower(),
             'punct': tokens[i],
             'prev-word-is-one-char': len(tokens[i-1]) == 1}

基于这一特征提取器，我们可以通过选择所有的标点符号创建一个加标签的特征集的列表，然后标注它们是否是边界标识符：

In [34]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
                for i in range(1, len(tokens)-1)
                if tokens[i] in '.?!']

使用这些特征集，我们可以训练和评估一个标点符号分类器：

In [35]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.936026936026936


使用这种分类器进行断句，我们只需检查每个标点符号，看它是否是作为一个边界标识符；在边界标识符处分割词列表。在2.1中的清单显示了如何可以做到这一点。

In [36]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

## 2.2 识别对话行为类型

处理对话时，将对话看作说话者执行的行为是很有用的。对于表述行为的陈述句这种解释是最直白的，例如"I forgive you"或"I bet you can't climb that hill"。但是问候、问题、回答、断言和说明都可以被认为是基于语言的行为类型。识别对话中言语下的对话行为是理解谈话的重要的第一步。

NPS聊天语料库，在1中的展示过，包括超过10,000个来自即时消息会话的帖子。这些帖子都已经被贴上15 种对话行为类型中的一种标签，例如“陈述”，“情感”，“yn问题”和“Continuer”。因此，我们可以利用这些数据建立一个分类器，识别新的即时消息帖子的对话行为类型。第一步是提取基本的消息数据。我们将调用xml_posts()来得到一个数据结构，表示每个帖子的XML注释：

In [37]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

下一步，我们将定义一个简单的特征提取器，检查帖子包含什么词：

In [38]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

最后，我们通过为每个帖子提取特征（使用post.get('class')获得一个帖子的对话行为类型）构造训练和测试数据，并创建一个新的分类器：

In [39]:
featuresets = [(dialogue_act_features(post.text), post.get('class'))
               for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.667


## 2.3 识别文字蕴含

识别文字蕴含（RTE）是判断文本T的一个给定片段是否蕴含着另一个叫做“假设”的文本（已经在5讨论过）。迄今为止，已经有4个RTE挑战赛，在那里共享的开发和测试数据会提供给参赛队伍。这里是挑战赛3开发数据集中的文本/假设对的两个例子。标签True表示蕴含成立，False表示蕴含不成立。
```py
    Challenge 3, Pair 34 (True)

        T: Parviz Davudi was representing Iran at a meeting of the Shanghai Co-operation Organisation (SCO), the fledgling association that binds Russia, China and four former Soviet republics of central Asia together to fight terrorism.

        H: China is a member of SCO.

    Challenge 3, Pair 81 (False)

        T: According to NC Articles of Organization, the members of LLC company are H. Nelson Beavers, III, H. Chester Beavers and Jennie Beavers Stewart.

        H: Jennie Beavers Stewart is a share-holder of Carolina Analytical Laboratory.
```
应当强调，文字和假设之间的关系并不一定是逻辑蕴涵，而是一个人是否会得出结论：文本提供了合理的证据证明假设是真实的。

我们可以把RTE当作一个分类任务，尝试为每一对预测True/False标签。虽然这项任务的成功做法似乎看上去涉及语法分析、语义和现实世界的知识的组合，RTE的许多早期的尝试使用粗浅的分析基于文字和假设之间的在词级别的相似性取得了相当不错的结果。在理想情况下，我们希望如果有一个蕴涵那么假设所表示的所有信息也应该在文本中表示。相反，如果假设中有的资料文本中没有，那么就没有蕴涵。

在我们的RTE 特征探测器（2.2）中，我们让词（即词类型）作为信息的代理，我们的特征计数词重叠的程度和假设中有而文本中没有的词的程度（由hyp_extra()方法获取）。不是所有的词都是同样重要的——命名实体，如人、组织和地方的名称，可能会更为重要，这促使我们分别为words和nes（命名实体）提取不同的信息。此外，一些高频虚词作为“停用词”被过滤掉。



In [40]:
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
    features['word_overlap'] = len(extractor.overlap('word'))
    features['word_hyp_extra'] = len(extractor.hyp_extra('word'))
    features['ne_overlap'] = len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

为了说明这些特征的内容，我们检查前面显示的文本/假设对34的一些属性：

In [42]:
rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
extractor = nltk.RTEFeatureExtractor(rtepair)
print(extractor.text_words)
print(extractor.hyp_words)
print(extractor.overlap('word'))
print(extractor.overlap('ne'))
print(extractor.hyp_extra('word'))

{'former', 'republics', 'Davudi', 'SCO', 'Russia', 'terrorism.', 'representing', 'Shanghai', 'Organisation', 'fight', 'at', 'binds', 'that', 'association', 'four', 'Parviz', 'central', 'together', 'meeting', 'Iran', 'operation', 'Asia', 'fledgling', 'China', 'was', 'Co', 'Soviet'}
{'China', 'SCO.', 'member'}
set()
{'China'}
{'member'}


这些特征表明假设中所有重要的词都包含在文本中，因此有一些证据支持标记这个为True。

nltk.classify.rte_classify模块使用这些方法在合并的RTE测试数据上取得了刚刚超过58％的准确率。这个数字并不是很令人印象深刻的，还需要大量的努力，更多的语言学处理，才能达到更好的结果。

# 3 评估

为了决定一个分类模型是否准确地捕捉了模式，我们必须评估该模型。评估的结果对于决定模型是多么值得信赖以及我们如何使用它是非常重要。评估也可以是一个有效的工具，用于指导我们在未来改进模型。
## 3.1 测试集

大多数评估技术为模型计算一个得分，通过比较它在测试集（或评估集）中为输入生成的标签与那些输入的正确标签。该测试集通常与训练集具有相同的格式。然而，测试集与训练语料不同是非常重要的：如果我们简单地重复使用训练集作为测试集，那么一个只记住了它的输入而没有学会如何推广到新的例子的模型会得到误导人的高分。

建立测试集时，往往是一个可用于测试的和可用于训练的数据量之间的权衡。对于有少量平衡的标签和一个多样化的测试集的分类任务，只要100个评估实例就可以进行有意义的评估。但是，如果一个分类任务有大量的标签或包括非常罕见的标签，那么选择的测试集的大小就要保证出现次数最少的标签至少出现50次。此外，如果测试集包含许多密切相关的实例——例如来自一个单独文档中的实例——那么测试集的大小应增加，以确保这种多样性的缺乏不会扭曲评估结果。当有大量已标注数据可用时，只使用整体数据的10％进行评估常常会在安全方面犯错。

选择测试集时另一个需要考虑的是测试集中实例与开发集中的实例的相似程度。这两个数据集越相似，我们对将评估结果推广到其他数据集的信心就越小。例如，考虑词性标注任务。在一种极端情况，我们可以通过从一个反映单一的文体（如新闻）的数据源随机分配句子，创建训练集和测试集：

In [43]:
import random
from nltk.corpus import brown
tagged_sents = list(brown.tagged_sents(categories='news'))
random.shuffle(tagged_sents)
size = int(len(tagged_sents) * 0.1)
train_set, test_set = tagged_sents[size:], tagged_sents[:size]

在这种情况下，我们的测试集和训练集将是非常相似的。训练集和测试集均取自同一文体，所以我们不能相信评估结果可以推广到其他文体。更糟糕的是，因为调用random.shuffle()，测试集中包含来自训练使用过的相同的文档的句子。如果文档中有相容的模式（也就是说，如果一个给定的词与特定词性标记一起出现特别频繁），那么这种差异将体现在开发集和测试集。一个稍好的做法是确保训练集和测试集来自不同的文件：

In [44]:
file_ids = brown.fileids(categories='news')
size = int(len(file_ids) * 0.1)
train_set = brown.tagged_sents(file_ids[size:])
test_set = brown.tagged_sents(file_ids[:size])

如果我们要执行更令人信服的评估，可以从与训练集中文档联系更少的文档中获取测试集：

In [45]:
train_set = brown.tagged_sents(categories='news')
test_set = brown.tagged_sents(categories='fiction')

如果我们在此测试集上建立了一个性能很好的分类器，那么我们完全可以相信它有能力很好的泛化到用于训练它的数据以外的数据。

## 3.2 准确度

用于评估一个分类最简单的度量是准确度，测量测试集上分类器正确标注的输入的比例。例如，一个名字性别分类器，在包含80个名字的测试集上预测正确的名字有60个，它有60/80 = 75％的准确度。nltk.classify.accuracy()函数会在给定的测试集上计算分类器模型的准确度：

In [46]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features
featuresets = [(dialogue_act_features(post.text), post.get('class'))
               for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: {:4.2f}'.format(nltk.classify.accuracy(classifier, test_set))) 

Accuracy: 0.67


解释一个分类器的准确性得分，考虑测试集中单个类标签的频率是很重要的。例如，考虑一个决定词bank每次出现的正确的词意的分类器。如果我们在金融新闻文本上评估分类器，那么我们可能会发现，金融机构的意思20个里面出现了19次。在这种情况下，95％的准确度也难以给人留下深刻印象，因为我们可以实现一个模型，它总是返回金融机构的意义。然而，如果我们在一个更加平衡的语料库上评估分类器，那里的最频繁的词意只占40％，那么95％的准确度得分将是一个更加积极的结果。（在2测量标注一致性程度时也会有类似的问题。）

## 3.4 混淆矩阵

当处理有3 个或更多的标签的分类任务时，基于模型错误类型细分模型的错误是有信息量的。一个混淆矩阵是一个表，其中每个cells [i,j]表示正确的标签i被预测为标签j的次数。因此，对角线项目（即cells |ii|）表示正确预测的标签，非对角线项目表示错误。在下面的例子中，我们为4中开发的一元标注器生成一个混淆矩阵：

In [47]:
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

In [48]:
def tag_list(tagged_sents):
    return [tag for sent in tagged_sents for (word, tag) in sent]
def apply_tagger(tagger, corpus):
    return [tagger.tag(nltk.tag.untag(sent)) for sent in corpus]
gold = tag_list(brown.tagged_sents(categories='editorial'))
test = tag_list(apply_tagger(t2, brown.tagged_sents(categories='editorial')))
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

    |                                         N                      |
    |      N      I      A      J             N             V      N |
    |      N      N      T      J      .      S      ,      B      P |
----+----------------------------------------------------------------+
 NN | <11.9%>  0.0%      .   0.2%      .   0.0%      .   0.2%   0.0% |
 IN |   0.0%  <9.0%>     .      .      .   0.0%      .      .      . |
 AT |      .      .  <8.6%>     .      .      .      .      .      . |
 JJ |   1.6%      .      .  <4.0%>     .      .      .   0.0%   0.0% |
  . |      .      .      .      .  <4.8%>     .      .      .      . |
NNS |   1.5%      .      .      .      .  <3.3%>     .      .   0.0% |
  , |      .      .      .      .      .      .  <4.4%>     .      . |
 VB |   0.9%      .      .   0.0%      .      .      .  <2.4%>     . |
 NP |   1.0%      .      .   0.0%      .      .      .      .  <1.9%>|
----+----------------------------------------------------------------+
(row =

这个混淆矩阵显示常见的错误，包括NN替换为了JJ（1.6%的词），NN替换为了NNS（1.5%的词）。注意点（.）表示值为0 的单元格，对角线项目——对应正确的分类——用尖括号标记。